In [7]:
# Requisitos:
# pip install langchain-openai langchain-core langchain langgraph pydantic requests numpy python-dotenv

import os, time, requests, numpy as np
from typing import Literal, TypedDict, Dict, List
from dotenv import load_dotenv; load_dotenv()
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import json

BINANCE_BASE = "https://api.binance.com"
SYMBOL = "BTCUSDT"

API_KEY = os.getenv("BINANCE_API_KEY", "")
API_SECRET = os.getenv("BINANCE_API_SECRET", "")

# Session con header de API key (Binance lo ignora en públicos, pero lo incluimos)
SESSION = requests.Session()
if API_KEY:
    SESSION.headers.update({"User-Agent": "Mozilla/5.0"}) 

# Hosts rotativos (api1..3 y mirror público)
BINANCE_HOSTS = [
    os.getenv("BINANCE_BASE", "https://api.binance.com"),
    "https://api1.binance.com",
    "https://api2.binance.com",
    "https://api3.binance.com",
    "https://data-api.binance.vision",
]    

def _get_with_fallback(path: str, params: dict, timeout=10):
    last_err = None
    for base in BINANCE_HOSTS:
        url = f"{base}{path}"
        try:
            r = SESSION.get(url, params=params, timeout=timeout)
            # Algunos WAF devuelven 451/403/5xx: probamos siguiente host
            if r.status_code >= 400:
                # 451=legal block, 403=forbidden, 429=rate limit
                if r.status_code in (451, 403, 429) or 500 <= r.status_code < 600:
                    last_err = requests.HTTPError(f"{r.status_code} for {url}", response=r)
                    continue
            r.raise_for_status()
            return r.json(), base
        except (requests.ConnectionError, requests.Timeout, requests.HTTPError) as e:
            last_err = e
            continue
    # Si ninguno funcionó:
    raise last_err or RuntimeError("Fallo de red al consultar Binance")

# ----------------------------
# Utils: fetch klines + EMA
# ----------------------------
def fetch_klines(symbol: str, interval: str, limit: int = 200):
    """Trae klines probando varios hosts (maneja 451/403/5xx). Devuelve (times, closes)."""
    data, used_base = _get_with_fallback(
        "/api/v3/klines", {"symbol": symbol, "interval": interval, "limit": limit}, timeout=12
    )
    # print(f"KLINES host usado: {used_base}")  # descomenta si querés ver el host elegido
    closes = [float(x[4]) for x in data]
    times = [int(x[0]) for x in data]
    return times, closes

def ema(series: List[float], period: int) -> np.ndarray:
    arr = np.array(series, dtype=float)
    k = 2 / (period + 1)
    out = np.zeros_like(arr)
    out[0] = arr[0]
    for i in range(1, len(arr)):
        out[i] = arr[i] * k + out[i-1] * (1 - k)
    return out

def crossover_signal(closes: List[float]) -> Literal["bull","bear","none"]:
    """Cruce EMA9 vs EMA21 en la última vela."""
    if len(closes) < 50:
        return "none"
    e9, e21 = ema(closes, 9), ema(closes, 21)
    prev, curr = e9[-2] - e21[-2], e9[-1] - e21[-1]
    eps = max(1e-6, 0.0001 * closes[-1])
    if prev < -eps and curr > eps:   # pasó de abajo a arriba
        return "bull"
    if prev >  eps and curr < -eps:  # pasó de arriba a abajo
        return "bear"
    return "none"

# ----------------------------
# Tools para el agente
# ----------------------------
@tool
def get_btc_signal(interval: Literal["1h","4h","1d"]="1h", limit: int = 200) -> str:
    """Devuelve el 'signal' (bull/bear/none) de BTCUSDT en {1h,4h,1d} por cruce EMA9/EMA21."""
    times, closes = fetch_klines(SYMBOL, interval, limit)
    sig = crossover_signal(closes)
    ts = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(times[-1]/1000))
    return f"{interval}:{sig} @ {ts} UTC (close={closes[-1]:.2f})"

TOOLS = [get_btc_signal]

# ----------------------------
# LLM (razonamiento/explicación)
# ----------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un agente de trading profesional.
Usa herramientas para señales por timeframe (1h, 4h, 1d).
Regla:
- 3/3 bull  -> NOTIFICAR: COMPRAR
- 3/3 bear  -> NOTIFICAR: VENDER
- Si 2/3 coinciden -> OBSERVAR
- Otro caso -> SIN SEÑAL
Incluye breve razonamiento"""),
    ("human", "{input}"),
MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent_runnable = create_tool_calling_agent(llm, TOOLS, prompt)
agent = AgentExecutor(agent=agent_runnable, tools=TOOLS, verbose=False)

# ----------------------------
# LangGraph
# ----------------------------
class GraphState(TypedDict):
    input: str
    tool_logs: List[str]
    signals: Dict[str, str]
    decision: str
    explanation: str

def agent_node(state: GraphState) -> GraphState:
    # Aseguramos consultar los 3 intervalos (independiente del output del LLM)
    tool_logs, signals = [], {}
    for itv in ["1h","4h","1d"]:
        res = get_btc_signal.invoke({"interval": itv})
        tool_logs.append(res)
        tag, sig = res.split(":")[0], res.split(":")[1].split(" ")[0]
        signals[tag] = sig

    vals = list(signals.values())
    if all(v == "bull" for v in vals):
        decision = "COMPRAR"
    elif all(v == "bear" for v in vals):
        decision = "VENDER"
    else:
        decision = "OBSERVAR" if (vals.count("bull")==2 or vals.count("bear")==2) else "SIN SEÑAL"

    expl_prompt = ChatPromptTemplate.from_messages([
        ("system", "Resume señales y explica la decisión en 2-3 líneas. Incluye disclaimer."),
        ("human", "Señales: {signals_json}. Decisión: {decision}.")
    ])
    explanation = (expl_prompt | llm).invoke({
        "signals_json": json.dumps(signals, ensure_ascii=False),
        "decision": decision
    }).content    

    return {**state, "tool_logs": tool_logs, "signals": signals,
            "decision": decision, "explanation": explanation}

graph = StateGraph(GraphState)
graph.add_node("agent_node", agent_node)
graph.add_edge("agent_node", END)
graph.set_entry_point("agent_node")
memory = MemorySaver()
app = graph.compile(checkpointer=memory)

if __name__ == "__main__":
    initial = {"input": "Evaluar señales BTC (1h/4h/1d)",
               "tool_logs": [], "signals": {}, "decision": "", "explanation": ""}
    result = app.invoke(
        initial,
        config={"configurable": {"thread_id": "btc-signals-run-1"}}
    )
    print("TOOL LOGS:")
    for l in result["tool_logs"]:
        print(" -", l)
    print("SEÑALES:", result["signals"])
    print("DECISIÓN:", result["decision"])
    print("EXPLICACIÓN:", result["explanation"])


TOOL LOGS:
 - 1h:none @ 2025-09-27 04:00:00 UTC (close=109580.00)
 - 4h:none @ 2025-09-27 04:00:00 UTC (close=109580.01)
 - 1d:none @ 2025-09-27 00:00:00 UTC (close=109580.00)
SEÑALES: {'1h': 'none', '4h': 'none', '1d': 'none'}
DECISIÓN: SIN SEÑAL
EXPLICACIÓN: La decisión es "SIN SEÑAL" porque no se identificaron indicios de movimiento en ninguna de las temporalidades analizadas (1 hora, 4 horas y 1 día). Esto sugiere que no hay oportunidades claras para operar en este momento. 

Disclaimer: Esta información no constituye asesoramiento financiero y se debe considerar en el contexto de un análisis más amplio.
